In [14]:
import os
import sys
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())
print(f"Aktuelles Arbeitsverzeichnis: {os.getcwd()}")

Aktuelles Arbeitsverzeichnis: c:\Users\hp\OneDrive\Desktop\DBU\wai81-ai-theory\ml_picture_recognition


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models.video import r3d_18
from utils.dataloader import HockeyDataset
from utils.transforms import train_transform, val_transform
from sklearn.metrics import f1_score
from torch.cuda.amp import autocast, GradScaler


In [16]:
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())
print(f"Aktuelles Arbeitsverzeichnis: {os.getcwd()}")

Aktuelles Arbeitsverzeichnis: c:\Users\hp\OneDrive\Desktop\DBU\wai81-ai-theory\ml_picture_recognition


In [17]:
# --- Pfade und Parameter ---
TRAIN_CSV = 'data/labels_train.csv'
VAL_CSV = 'data/labels_val.csv'
FRAMES_ROOT = 'data/train_frames'

BATCH_SIZE = 2  # R3D braucht viel GPU-Speicher, kleiner Batch!
NUM_WORKERS = 4
NUM_CLASSES = 4
LEARNING_RATE = 1e-4
EPOCHS = 30
PATIENCE = 5

In [18]:
# --- Dataset & DataLoader ---
train_dataset = HockeyDataset(csv_file=TRAIN_CSV, frames_root=FRAMES_ROOT, transform=train_transform)
val_dataset = HockeyDataset(csv_file=VAL_CSV, frames_root=FRAMES_ROOT, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)


In [19]:
# --- Modell ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = r3d_18(weights='KINETICS400_V1')
model.stem[0] = nn.Conv3d(3, 64, kernel_size=(3,7,7), stride=(1,2,2), padding=(1,3,3), bias=False)  # Input-Anpassung
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)  # 4 Klassen
model = model.to(device)


In [20]:
# --- Loss & Optimizer ---
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [21]:
# --- Training Setup ---
best_val_f1 = 0
early_stopping_counter = 0

train_losses = []
train_f1_scores = []
val_losses = []
val_f1_scores = []


scaler = GradScaler()

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    all_preds, all_targets = [], []

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.float().to(device)

        inputs = inputs.permute(0, 2, 1, 3, 4)  # GANZ WICHTIG!! (B, T, C, H, W) -> (B, C, T, H, W)

        optimizer.zero_grad()

        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).float()
        all_preds.append(preds.cpu())
        all_targets.append(labels.cpu())

    train_loss = running_loss / len(train_loader)
    train_f1 = f1_score(torch.cat(all_targets), torch.cat(all_preds), average="macro", zero_division=0)

    train_losses.append(train_loss)
    train_f1_scores.append(train_f1)

    # --- Validation ---
    model.eval()
    val_loss = 0.0
    val_preds, val_targets = [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.permute(0, 2, 1, 3, 4)
            inputs = inputs.to(device)
            labels = labels.float().to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_preds.append((torch.sigmoid(outputs) > 0.5).cpu())
            val_targets.append(labels.cpu())

    val_loss /= len(val_loader)
    val_f1 = f1_score(torch.cat(val_targets), torch.cat(val_preds), average="macro", zero_division=0)

    val_losses.append(val_loss)
    val_f1_scores.append(val_f1)

    # --- Fortschritt ausgeben ---
    print(f"Epoch {epoch+1}: Train Loss {train_loss:.4f}, F1 {train_f1:.2f} | Val Loss {val_loss:.4f}, F1 {val_f1:.2f}")

    # --- Early Stopping ---
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        early_stopping_counter = 0
        torch.save(model.state_dict(), 'models/best_r3d18.pth')
    else:
        early_stopping_counter += 1
        print(f"Early Stopping Counter: {early_stopping_counter}/{PATIENCE}")
        if early_stopping_counter >= PATIENCE:
            print("⏹️ Early stopping triggered.")
            break

C:\Users\hp\AppData\Local\Temp\ipykernel_5056\1161651495.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\hp\AppData\Local\Temp\ipykernel_5056\1161651495.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1: Train Loss 0.5983, F1 0.35 | Val Loss 0.5850, F1 0.35


C:\Users\hp\AppData\Local\Temp\ipykernel_5056\1161651495.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 2: Train Loss 0.5293, F1 0.39 | Val Loss 0.5024, F1 0.48


C:\Users\hp\AppData\Local\Temp\ipykernel_5056\1161651495.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


...

Epoch 24: Train Loss 0.1589, F1 0.93 | Val Loss 0.4042, F1 0.68
Early Stopping Counter: 3/5


C:\Users\hp\AppData\Local\Temp\ipykernel_5056\1161651495.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 25: Train Loss 0.1443, F1 0.94 | Val Loss 0.3670, F1 0.66
Early Stopping Counter: 4/5


C:\Users\hp\AppData\Local\Temp\ipykernel_5056\1161651495.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 26: Train Loss 0.1268, F1 0.94 | Val Loss 0.3863, F1 0.71
Early Stopping Counter: 5/5
⏹️ Early stopping triggered.
